In [ ]:
import pandas as pd
import sklearn
import sklearn.ensemble
import csv
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from google.colab import files
#########
#uploaded= files.upload()

In [ ]:
#These are linux commands to download data files. They will be saved in the current directory
#Run these commands one time
!wget https://github.com/ahmedmk323/CS301-Project/blob/master/csv/positive.review
!wget https://github.com/ahmedmk323/CS301-Project/blob/master/csv/negative.review

--2021-04-26 14:52:42--  https://github.com/ahmedmk323/CS301-Project/blob/master/csv/negative.review
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘negative.review.1’

negative.review.1       [ <=>                ] 104.85K  --.-KB/s    in 0.02s   

2021-04-26 14:52:42 (4.79 MB/s) - ‘negative.review.1’ saved [107369]



# Data Extarction Function

In [ ]:
features=["review","unique_id","unique_num""rating","title", "review_text","class_"]
def data_extraction(f_name, c): #c is an int that identfies the class value 0 or 1
  file= open(f_name, 'r', encoding='latin-1')
  reader= csv.reader(file,delimiter='\n')
  lst= []
  buffer=""
  flag= False
  index=-1
  for row in reader:
    if (row == []):
      continue
    slash= row[0][1:2] #To check whether slash exists or not
    strip_w= row[0].strip("<>")
    
    if (slash == '/' and ((strip_w.strip('/')) in features)):
      if ((strip_w) =="/review"):
        lst[index].append(c)

      if (buffer != '' and (buffer not in lst[index])):
        lst[index].append(buffer)
      buffer=""
      flag= False
      continue
    if (strip_w == "review"):
      index+=1
      lst.append([])
    elif (strip_w in features):
      flag=True
      buffer=""
      continue
    if (flag):
      buffer+= row[0]
  return lst

In [ ]:
  
 #positvie.REVIEW raw data file
 # negative.REVIEW raw data file
p_data= data_extraction('./positive.review', 1) 
n_data= data_extraction('./negative.review', 0)

df0=pd.DataFrame(p_data,columns=(features[1::])) #Dataframe 1 to load positve reviews data
df1=pd.DataFrame(n_data,columns=(features[1::])) #Dataframe 2 to load negative reviews data
frames=[df0,df1]
main_df=pd.concat(frames) #Merging the two dataframes  
main_df= main_df.dropna() #Dropping rows that cointain Nan values
main_df= main_df.sample(frac=1) #Randomizing rows in the dataframe

X=main_df.review_text
y=main_df.class_

X_train, X_test, y_train,y_test= train_test_split(X,y, test_size=0.2) #splitting training and test data

In [ ]:
vectorizer = sklearn.feature_extraction.text.TfidfVectorizer(lowercase=False)
train_vectors = vectorizer.fit_transform(X_train)
test_vectors = vectorizer.transform(X_test)


# USING RANDOM FOREST CLASSIFIER

In [ ]:
rf = sklearn.ensemble.RandomForestClassifier(n_estimators=500)
rf.fit(train_vectors,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
pred = rf.predict(test_vectors)
sklearn.metrics.f1_score(y_test, pred, average='binary')
print(sklearn.metrics.classification_report(np.rint(pred),y_test))

              precision    recall  f1-score   support

         0.0       0.88      0.66      0.75       116
         1.0       0.58      0.85      0.69        65

    accuracy                           0.72       181
   macro avg       0.73      0.75      0.72       181
weighted avg       0.77      0.72      0.73       181

